In [ ]:
import girderformindlogger
import girder_client as gc
import pandas as pd
import numpy as np
import simplejson

In [2]:
girder = gc.GirderClient(host="localhost", port=8080)

# Plan

### Basics 

- [x] create a new user (userA)
- [x] get userA's applets and make sure the list is empty
- [x] userA creates an applet with an activity-set-url (url TBD)
- [x] get userA's applets and make sure we have the applet they just created
- [x] create userB
- [x] userA invites userB to the applet
- [x] make sure userB has an invite
- [x] userB accepts their invite
- [x] list userB's applets
- [x] userA invites userC (by email) to the applet
- [x] check that the applet user table has userC pending
- [x] create userC with the same email
- [x] check that userC has an invite
- [x] accept userC's applet invite
- [x] check userC's applets
- [x] userA, B and C submit responses to each activity in the applet
- [x] user B removes the applet without removing data
- [x] user C removes the applet AND removes data
- [x] delete userB and userC
- [x] userA deactivites their applet
- [x] delete userA
- [x] refresh an applet
- [x] add a schedule to an applet
- [x] add test for nested sub-activities

### Backend TODOs

- [x] userA can see user's B & C's data
- [x] get responses for each user in the last 7 days

# Create a new user

In [3]:
def testCreateUser(email=None):
    randomUser = np.random.randint(1000000)
    firstName = 'test'
    lastName = 'user'
    # NOTE: girder makes login and email lowercase!!!
    login = 'testuser{}'.format(randomUser)
    if not email:
        email = 'testemail{}@testemail.com'.format(randomUser)
    password = 'password'
    createUser = girder.post('user', parameters=dict(firstName=firstName,
                                        lastName=lastName,
                                        login=login,
                                        email=email,
                                        password=password
                                       )
               )
    # assert 'authToken' in createUser.keys(), 'user has no token, {}'.format(createUser)
    assert createUser['email'] == email, 'email does not match, {} {}'.format(createUser['email'], email)
    assert createUser['firstName'] == firstName, 'firstName does not match'
    assert createUser['lastName'] == lastName, 'lastName does not match'
    assert createUser['login'] == login, 'login does not match, {} {}'.format(createUser['login'], login)
    assert createUser['admin'] == False, 'user is an admin'
    assert createUser['_modelType'] == 'user', 'model is not user'
    assert createUser['public'] == False, 'user is public!'

    return createUser

In [4]:
def authenticate(user):
    girder.authenticate(username=user['login'], password='password')

In [5]:
userA = testCreateUser()
userB = testCreateUser()

#authenticate(userA)

# Get applets when the user doesn't have any

In [6]:
def getAppletsNewUser(new_user):
    girder.authenticate(username=new_user['login'], password='password')
    appletList = girder.get('user/applets')
    assert len(appletList) == 0, 'a new user should have an empty list of applets. we get {}'.format(appletList)
    return 1

In [7]:
getAppletsNewUser(userA)

1

# get Applet by ID

want to check recently added applet here

In [8]:
def getAppletById(user, ar):
    girder.authenticate(username=user['login'], password='password')
    res = girder.get('applet/{}'.format(ar['_id']))
    assert res['applet']['_id'].split('applet/')[1] == ar['_id'], 'applet ids are not the same'
    return 1

# Add an applet

activity set info is below:

In [9]:
activitySetUrl = 'https://raw.githubusercontent.com/ReproNim/schema-standardization/master/activity-sets/ema-hbn/ema-hbn_schema'

act1 = 'https://raw.githubusercontent.com/ReproNim/schema-standardization/master/activities/EmaHBNEvening/ema_evening_schema'
act2 = 'https://raw.githubusercontent.com/ReproNim/schema-standardization/master/activities/EmaHBNMorning/ema_morning_schema'

act1Item = 'https://raw.githubusercontent.com/ReproNim/schema-standardization/master/activities/EmaHBNEvening/items/good_bad_day'
act2Item = 'https://raw.githubusercontent.com/ReproNim/schema-standardization/master/activities/EmaHBNMorning/items/sleeping_aids'

In [10]:
nestedActivitySet = 'https://raw.githubusercontent.com/ReproNim/schema-standardization/master/activity-sets/pediatric-screener/pediatric-screener_schema'

nact1 = 'https://raw.githubusercontent.com/ReproNim/schema-standardization/master/activities/PediatricScreener-Parent/pediatric_screener_parent_schema'
nact2 = 'https://raw.githubusercontent.com/ReproNim/schema-standardization/master/activities/PediatricScreener-SelfReport/pediatric_screener_selfreport_schema'

nact1Item = 'https://raw.githubusercontent.com/ReproNim/schema-standardization/master/activities/PediatricScreener-Parent/items/fidgety'
nact2Item = 'https://raw.githubusercontent.com/ReproNim/schema-standardization/master/activities/PediatricScreener-SelfReport/items/having_less_fun'




In [11]:
def addApplet(new_user, activitySetUrl):
    girder.authenticate(username=new_user['login'], password='password')

    # TODO: create an activity-set that JUST for testing.
    # make sure it has all the weird qualities that can break


    # for now, lets do the mindlogger demo
    activitySetUrl = activitySetUrl
    randomAS = np.random.randint(1000000)
    ar = girder.post('applet', parameters=dict(activitySetUrl = activitySetUrl, name='testActivitySet{}'.format(randomAS)))

    assert ar['_id'], 'there is no ID!'
    assert ar['meta']['activitySet']['url'] == activitySetUrl, 'the URLS do not match! {} {}'.format(ar['meta']['activitySet']['url'], activitySetUrl)
    
    assert getAppletById(new_user, ar) == 1, 'something wrong with getAppletById'
    return ar

In [12]:
# userA_applets = addApplet(userA, activitySetUrl)
userA_applets = addApplet(userA, nestedActivitySet)

In [13]:
userA_applets['roles']['reviewer']['groups']

[{'id': '5d8a55f46ea03b31fd5a39fa', 'subject': None}]

In [14]:
def getAppletsUser(user, n=1):
    girder.authenticate(username=user['login'], password='password')
    appletList = girder.get('user/applets')
    assert len(appletList) == n, 'a new user should have {} applets. we get {}'.format(n, len(appletList))
    return 1

In [15]:
getAppletsUser(userA, 1)

1

### test the expanded applet with nested sub-activities

In [16]:
# TODO: specify the applet instead of using the first one.
def getExpandedApplet(userA):
    authenticate(userA)
    expandedApplet = girder.get('user/applets')[0]
    return expandedApplet

In [17]:
expApplet = getExpandedApplet(userA)

In [18]:
assert len(expApplet['items']) == 59, 'there should be a total of 59 items for the whole applet'

In [19]:
assert len(expApplet['activities']) == 8, 'there should be 8 activities for the whole applet, but some are sub-activities'

### Refresh an applet

In [20]:
def refreshApplet(userA, userA_applets):
    authenticate(userA)
    appletId = userA_applets['_id']
    refreshResp = girder.get('applet/{}?refreshCache=true'.format(appletId))
    # TODO: somehow check that the refresh actually worked?
    return refreshResp

In [21]:
refreshed = refreshApplet(userA, userA_applets)

In [22]:
refreshed['applet']['url']

'https://raw.githubusercontent.com/ReproNim/schema-standardization/master/activity-sets/pediatric-screener/pediatric-screener_schema'

### add a schedule

In [23]:
def addSchedule(userA, userA_applets):
    scheduleString = """{"type":2,"size":1,"fill":true,"minimumSize":0,"repeatCovers":true,"listTimes":false,"eventsOutside":false,"updateRows":false,"updateColumns":false,"around":1567321200000,"events":[{"data":{"title":"EMA: Morning","description":"","location":"","color":"#673AB7","forecolor":"#ffffff","calendar":"","busy":true,"icon":"","URI":"http://repronim.org/schema-standardization/activity-sets/mindlogger-demo/mindlogger-demo_schema","notifications":[{"start":"09:00","end":null,"random":false,"notifyIfIncomplete":false}],"useNotifications":true},"schedule":{}}]}
    """
    schedule = simplejson.loads(scheduleString)
    authenticate(userA)
    appletId = userA_applets['_id']
    putResp = girder.put('applet/{}/constraints/'.format(appletId), data=dict(schedule=simplejson.dumps(schedule)))
    assert putResp['applet']['schedule'] == schedule

In [24]:
addSchedule(userA, userA_applets)

# Invite userB to userA's applet

In [25]:
def inviteUserToApplet(userA, userA_applets, userB):
    groupId = userA_applets['roles']['user']['groups'][0]['id']
    authenticate(userA)
    inviteResp = girder.post('group/{}/invitation'.format(groupId), {
        "email": userB['email']
    })
    assert inviteResp['public'] == False, 'invite is public!'
    assert len(inviteResp['access']['users']), 'ths user was not added'
    return inviteResp

In [26]:
inviteB = inviteUserToApplet(userA, userA_applets, userB)

## Get a group's pending invites and check for a user

In [27]:
def checkInvitesForUser(userA, userA_applets, userB):
    authenticate(userA)
    groupId = userA_applets['roles']['user']['groups'][0]['id']
    pendingInvites = girder.get('group/{}/invitation'.format(groupId))
    value = False
    for invite in pendingInvites:
        if invite['email'] == userB['email']:
            value = True
            break
    assert value, "email not in list of pending invites, {}".format(pendingInvites)
    return 1

In [28]:
checkInvitesForUser(userA, userA_applets, userB)

1

### Accept an applet invite

In [29]:
def checkForInvite(userB, userA_applets):
    authenticate(userB)
    pendingInvitesForUser = girder.get('user/invites')
    groupId = userA_applets['roles']['user']['groups'][0]['id']
    assert len(pendingInvitesForUser), "this user has no invites"
    assert pendingInvitesForUser[0]['_id'] == groupId, "this user doesn't have the invite you expect"
    return groupId

In [30]:
def acceptAppletInvite(userB, userA_applets):
    groupId = checkForInvite(userB, userA_applets)
    authenticate(userB)
    resp = girder.post('group/{}/member'.format(groupId))
    assert resp['_modelType'] == 'group', "something weird about response, {}".format(resp)
    return 1

In [31]:
acceptAppletInvite(userB, userA_applets)

1

get a list of applet users (manager)

In [32]:
def getUserTable(userA, userA_applets):
    authenticate(userA)
    appletUsers = girder.get('applet/{}/users'.format(userA_applets['_id']))
    return appletUsers

check that userB has an applet

In [33]:
getAppletsUser(userB, 1)

1

### Invite someone who does not have an account yet

In [34]:
userCemail = 'randomuserc{}@test.com'.format(np.random.randint(1000000))
inviteC = inviteUserToApplet(userA, userA_applets, {'email': userCemail})

In [35]:
inviteC

{'_accessLevel': 2,
 '_id': '5d8a55f46ea03b31fd5a39f7',
 '_modelType': 'group',
 'access': {'users': [{'flags': [],
    'id': '5d8a55e86ea03b31fd5a39ef',
    'level': 2}]},
 'created': '2019-09-24T17:44:20.394000+00:00',
 'description': '',
 'name': 'Default testActivitySet492453 (5d8a55f46ea03b31fd5a39f6) Users',
 'openRegistration': False,
 'public': False,
 'requests': [],
 'updated': '2019-09-24T17:45:31.339680+00:00'}

In [36]:
def checkAppletUserTableForUser(userA, userA_applets, userCemail):
    ut = getUserTable(userA, userA_applets)
    assert len(list(filter(lambda x: x['email'] == userCemail, ut))) == 1
    return ut

In [37]:
checkAppletUserTableForUser(userA, userA_applets, userCemail)

[{'_id': '5d8a563b6ea03b31fd5a3b8a',
  'email': 'randomuserc405984@test.com',
  'groups': [{'_id': '5d8a55f46ea03b31fd5a39f7',
    'name': 'Default testActivitySet492453 (5d8a55f46ea03b31fd5a39f6) Users',
    'role': 'user',
    'status': 'pending'}]},
 {'_id': '5d8a55e86ea03b31fd5a39ef',
  'email': 'testemail380809@testemail.com',
  'groups': [{'_id': '5d8a55f46ea03b31fd5a39f7',
    'name': 'Default testActivitySet492453 (5d8a55f46ea03b31fd5a39f6) Users',
    'role': 'user',
    'status': 'active'}]},
 {'_id': '5d8a55e86ea03b31fd5a39f2',
  'email': 'testemail582186@testemail.com',
  'groups': [{'_id': '5d8a55f46ea03b31fd5a39f7',
    'name': 'Default testActivitySet492453 (5d8a55f46ea03b31fd5a39f6) Users',
    'role': 'user',
    'status': 'active'}]},
 {'_id': '5d8a55e86ea03b31fd5a39ef',
  'email': 'testemail380809@testemail.com',
  'groups': [{'_id': '5d8a55f46ea03b31fd5a39f8',
    'name': 'Default testActivitySet492453 (5d8a55f46ea03b31fd5a39f6) Editors',
    'role': 'editor',
    '

create that person's account

In [38]:
userC = testCreateUser(userCemail)

and check if they have an invite

In [39]:
checkInvitesForUser(userA, userA_applets, userC)

1

In [40]:
checkForInvite(userC, userA_applets)

'5d8a55f46ea03b31fd5a39f7'

In [41]:
acceptAppletInvite(userC, userA_applets)

1

check that userC has applets

In [42]:
getAppletsUser(userC, 1)

1

# Post a response

In [43]:
def postResponse(userA, actURI, itemURI):
    authenticate(userA)
    expandedApplet = girder.get('user/applets')[0]
    appletId = expandedApplet['applet']['_id'].split('/')[1]
    a = expandedApplet['activities'][actURI]
    activityId = a['_id'].split('/')[1]
    # itemURI = itemURI
    response = {}
    response[itemURI] = np.random.randint(2)

    resp = girder.post('response/{}/{}'.format(appletId, activityId),
                       data={'metadata': simplejson.dumps(
                       {
                           'activity': activityId,
                           'applet': appletId,
                           'responses': response
                       }
                       )})
    assert resp['_id'], 'response is weird and does not have an id'
    assert 'activity' in resp['meta'].keys(), 'response does not have an activity'
    assert 'applet' in resp['meta'].keys(), 'response does not have an applet'
    assert 'responses' in resp['meta'].keys(), 'response does not have an response'
    return resp

In [44]:
# postResponse(userA, act1, act1Item)
# postResponse(userA, act2, act2Item)

# Note: posting the response under the top-level activity, not the sub-activity.

postResponse(userA, nact1, nact1Item)
postResponse(userA, nact2, nact2Item)

for i in range(5):

    postResponse(userB, nact1, nact1Item)
    postResponse(userB, nact2, nact2Item)

    postResponse(userC, nact1, nact1Item)
    postResponse(userC, nact2, nact2Item)

# postResponse(userB)
# postResponse(userC)

# As userA, get all data from the applet

In [101]:
def getDataForApplet(userA, userA_applets):
    authenticate(userA)
    appletId = userA_applets['_id']
    resp = girder.get('response', parameters={
        'applet': appletId
    })
    return resp


resp = getDataForApplet(userA, userA_applets)
len(resp)

22

In [103]:
df = pd.DataFrame(resp)
df.head()

,itemURI,schema:endDate,schema:startDate,userId,value
0,https://raw.githubusercontent.com/ReproNim/sch...,2019-09-24T17:45:50.725000,2019-09-24T17:45:50.725000,randomuserc405984@test.com,0
1,https://raw.githubusercontent.com/ReproNim/sch...,2019-09-24T17:45:50.378000,2019-09-24T17:45:50.378000,randomuserc405984@test.com,0
2,https://raw.githubusercontent.com/ReproNim/sch...,2019-09-24T17:45:50.019000,2019-09-24T17:45:50.019000,testemail582186@testemail.com,1
3,https://raw.githubusercontent.com/ReproNim/sch...,2019-09-24T17:45:49.671000,2019-09-24T17:45:49.671000,testemail582186@testemail.com,0
4,https://raw.githubusercontent.com/ReproNim/sch...,2019-09-24T17:45:49.320000,2019-09-24T17:45:49.320000,randomuserc405984@test.com,1


### Make userC a reviewer, and then make sure the userId is not an email address

In [80]:
authenticate(userA)
reviewerGroupId = userA_applets['roles']['reviewer']['groups'][0]['id']
userCReviewerInvite = girder.post('group/{}/invitation'.format(reviewerGroupId), {'email': userC['email']})

In [81]:
userCReviewerInvite

{'_accessLevel': 2,
 '_id': '5d8a55f46ea03b31fd5a39fa',
 '_modelType': 'group',
 'access': {'groups': [],
  'users': [{'email': 'testemail380809@testemail.com',
    'flags': [],
    'id': '5d8a55e86ea03b31fd5a39ef',
    'level': 2,
    'login': 'testuser380809',
    'name': 'test user'}]},
 'created': '2019-09-24T17:44:20.411000+00:00',
 'description': '',
 'name': 'Default testActivitySet492453 (5d8a55f46ea03b31fd5a39f6) Reviewers',
 'openRegistration': False,
 'public': True,
 'requests': [],
 'updated': '2019-09-24T17:44:20.411000+00:00'}

In [82]:
authenticate(userC)
girder.post('group/{}/member'.format(reviewerGroupId))

{'_accessLevel': 0,
 '_id': '5d8a55f46ea03b31fd5a39fa',
 '_modelType': 'group',
 'name': 'Default testActivitySet492453 (5d8a55f46ea03b31fd5a39f6) Reviewers'}

In [83]:
getUserTable(userA, userA_applets)

[{'_id': '5d8a55e86ea03b31fd5a39ef',
  'email': 'testemail380809@testemail.com',
  'groups': [{'_id': '5d8a55f46ea03b31fd5a39f7',
    'name': 'Default testActivitySet492453 (5d8a55f46ea03b31fd5a39f6) Users',
    'role': 'user',
    'status': 'active'}]},
 {'_id': '5d8a55e86ea03b31fd5a39f2',
  'email': 'testemail582186@testemail.com',
  'groups': [{'_id': '5d8a55f46ea03b31fd5a39f7',
    'name': 'Default testActivitySet492453 (5d8a55f46ea03b31fd5a39f6) Users',
    'role': 'user',
    'status': 'active'}]},
 {'_id': '5d8a563b6ea03b31fd5a3b8b',
  'email': 'randomuserc405984@test.com',
  'groups': [{'_id': '5d8a55f46ea03b31fd5a39f7',
    'name': 'Default testActivitySet492453 (5d8a55f46ea03b31fd5a39f6) Users',
    'role': 'user',
    'status': 'active'}]},
 {'_id': '5d8a55e86ea03b31fd5a39ef',
  'email': 'testemail380809@testemail.com',
  'groups': [{'_id': '5d8a55f46ea03b31fd5a39f8',
    'name': 'Default testActivitySet492453 (5d8a55f46ea03b31fd5a39f6) Editors',
    'role': 'editor',
    's

#### make sure that userC can't see the email address

In [96]:
resp = getDataForApplet(userC, userA_applets)

import pandas as pd

df = pd.DataFrame(resp)
df.head()

,itemURI,schema:endDate,schema:startDate,userId,value
0,https://raw.githubusercontent.com/ReproNim/sch...,2019-09-24T17:45:50.725000,2019-09-24T17:45:50.725000,99f8d703aaf854a5536e26692f354af2,0
1,https://raw.githubusercontent.com/ReproNim/sch...,2019-09-24T17:45:50.378000,2019-09-24T17:45:50.378000,99f8d703aaf854a5536e26692f354af2,0
2,https://raw.githubusercontent.com/ReproNim/sch...,2019-09-24T17:45:50.019000,2019-09-24T17:45:50.019000,408e51b1ba3eb8b15df9a89102537c1f,1
3,https://raw.githubusercontent.com/ReproNim/sch...,2019-09-24T17:45:49.671000,2019-09-24T17:45:49.671000,408e51b1ba3eb8b15df9a89102537c1f,0
4,https://raw.githubusercontent.com/ReproNim/sch...,2019-09-24T17:45:49.320000,2019-09-24T17:45:49.320000,99f8d703aaf854a5536e26692f354af2,1


### get last 7 days of responses

In [53]:
authenticate(userA)
appletId = userA_applets['_id']
last7 = girder.get('response/last7Days/{}'.format(appletId))
last7

{'responses': {'https://raw.githubusercontent.com/ReproNim/schema-standardization/master/activities/PediatricScreener-Parent/items/fidgety': [{'date': '2019-09-24',
    'value': 0}],
  'https://raw.githubusercontent.com/ReproNim/schema-standardization/master/activities/PediatricScreener-SelfReport/items/having_less_fun': [{'date': '2019-09-24',
    'value': 1}]}}

### Remove an applet without removing data

In [ ]:
def removeApplet(userB, userA_applets):
    authenticate(userB)
    groupId = userA_applets['roles']['user']['groups'][0]['id']
    girder.delete('group/{}/member?delete=false'.format(groupId))
    assert getAppletsUser(userB, 0), "user still has the applet that should be deleted."

removeApplet(userB, userA_applets)

### Remove an applet and the data

In [ ]:
def deleteApplet(userC, userA_applets):
    authenticate(userC)
    groupId = userA_applets['roles']['user']['groups'][0]['id']
    girder.delete('group/{}/member?delete=true'.format(groupId))
    assert getAppletsUser(userC, 0), "user still has the applet that should be deleted."

deleteApplet(userC, userA_applets)

### Deactivate an applet as admin

In [ ]:
def deactivateApplet(userA, userA_applets):
    authenticate(userA)
    girder.delete('applet/{}'.format(userA_applets['_id']))
    assert getAppletsUser(userA, 0), "user still has the applet that should be deleted."

deactivateApplet(userA, userA_applets)

# Delete a user

In [ ]:
def testDeleteUser(new_user):

    girder.authenticate(username=new_user['login'], password='password')

    deleteUser = girder.delete('user/{}'.format(new_user['_id']))

    assert deleteUser['message'] == 'Deleted user {}.'.format(new_user['login']), "{} does not equal {}".format(deleteUser['message'], 'Deleted user {}'.format(new_user['login']))
    
    return 1

In [ ]:
testDeleteUser(userA)
testDeleteUser(userB)
testDeleteUser(userC)